# Building a Chatbot with Conversation History

In this notebook, we will build a chatbot powered by a large language model (LLM). The main goal is to create a chatbot that can:

- **Understand user messages** and respond naturally.
- **Remember previous interactions** using a conversation history mechanism.
- **Support multi-turn conversations**, so the chatbot can refer back to earlier messages.

## Key Concepts Covered

### 1. LLM Integration
We use `ChatGroq` from LangChain to generate responses based on user input.

### 2. Message History
By storing all previous messages in a session, the chatbot can maintain context across multiple turns.

### 3. Prompt Templates
We define system instructions and message placeholders to structure user input for the model.

### 4. Conversation Management
We trim and manage message history to prevent the context from becoming too large. 



In [94]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
#groq_api_key



In [95]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
# model

In [96]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Nidhi and I am a AI Engineer")])

AIMessage(content="Hi Nidhi, it's nice to meet you!\n\nThat's awesome, being an AI Engineer is a really exciting field. What kind of work are you doing in AI? \n\nI'm always eager to learn more about what people are creating with AI.😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 22, 'total_tokens': 84, 'completion_time': 0.112727273, 'prompt_time': 0.001342839, 'queue_time': 0.25121576, 'total_time': 0.114070112}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--903cba19-ebe3-4b0d-bfa2-942c0db64dea-0', usage_metadata={'input_tokens': 22, 'output_tokens': 62, 'total_tokens': 84})

In [97]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Nidhi and I am a  AI Engineer"),
        AIMessage(content="Hello Nidhi! It's nice to meet you. \n\nAs a Nidhi AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Nidhi, and you are an AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 100, 'total_tokens': 136, 'completion_time': 0.065454545, 'prompt_time': 0.00273696, 'queue_time': 0.249981728, 'total_time': 0.068191505}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--cc2f6cdd-a599-4041-8b98-1203abbffbff-0', usage_metadata={'input_tokens': 100, 'output_tokens': 36, 'total_tokens': 136})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [98]:
!pip install langchain_community


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [100]:
config={"configurable":{"session_id":"chat1"}}

In [101]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Nidhi and I am a  AI Engineer")],
    config=config
)

In [102]:
response.content

"Hi Nidhi, it's nice to meet you!  \n\nThat's fascinating! As an AI myself, I'm always interested in meeting other people who work in the field. What kind of AI engineering do you specialize in? \n\nDo you have any projects you're particularly excited about?\n"

In [103]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Nidhi, as you told me at the beginning of our conversation. 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 103, 'total_tokens': 136, 'completion_time': 0.06, 'prompt_time': 0.00289324, 'queue_time': 0.249223029, 'total_time': 0.06289324}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d14f2499-d41f-4664-8b90-b0aaa62ed630-0', usage_metadata={'input_tokens': 103, 'output_tokens': 33, 'total_tokens': 136})

In [104]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it!\n"

In [105]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Nidhi")],
    config=config1
)
response.content

"Hi Nidhi, it's nice to meet you!  \n\nWhat can I do for you today?\n"

In [106]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Nidhi! 😊  \n\nI remember that you told me a little while ago.  \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [107]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [108]:
chain.invoke({"messages":[HumanMessage(content="Hey My name is Nidhi")]
              
              })

AIMessage(content="Hello Nidhi, it's nice to meet you!\n\nI'm ready to answer your questions to the best of my ability.  \n\nWhat can I help you with today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 32, 'total_tokens': 76, 'completion_time': 0.08, 'prompt_time': 0.00149537, 'queue_time': 0.249440479, 'total_time': 0.08149537}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--923962f6-7049-48a1-a680-99f0e5a22e31-0', usage_metadata={'input_tokens': 32, 'output_tokens': 44, 'total_tokens': 76})

In [109]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is Nidhi")],
    config = config)

In [110]:
response

AIMessage(content="Hi Nidhi!  \n\nIt's nice to meet you. What can I do for you today? I'm happy to answer any questions you have to the best of my ability. 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 32, 'total_tokens': 77, 'completion_time': 0.081818182, 'prompt_time': 0.001484829, 'queue_time': 0.250221161, 'total_time': 0.083303011}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b31d41a0-fb03-43f8-95a2-89e0dd475422-0', usage_metadata={'input_tokens': 32, 'output_tokens': 45, 'total_tokens': 77})

In [111]:
response = with_message_history.invoke(
    [HumanMessage(content = "What is My name?")],
    config= config
)

In [112]:
response.content

'Your name is Nidhi.  You told me yourself when you first introduced yourself! 😊  \n\n\n'

In [113]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [114]:
response = chain.invoke({"messages":[HumanMessage(content="Hey My name is Nidhi")],"language": "Hindi"})
response.content

'नमस्ते निधि! मुझे बहुत खुशी है कि आप मुझसे बात कर रही हैं। \n\nआप मुझसे कोई भी सवाल पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगी कि आपको सही जवाब दूं। \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [115]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [116]:
config ={ "configurable": {"session_id":"chat4"}}
response = with_message_history.invoke({"messages":[HumanMessage(content="Hi i am nidhi")], "language" :"Hindi"},
                                       config = config)


In [117]:
response.content

'नमस्ते निधि! 😊 \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आप क्या जानना चाहती हैं? \n'

In [118]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What is My name?")], "language" :"Hindi"},
                                       config = config)
    
    


In [119]:
response.content

'आपका नाम निधि है। 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [120]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [121]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences! 🍦\n\nWhat's your favorite flavor? 😋\n"

In [122]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2".  😄  \n\n\n\n'

In [123]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [124]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [125]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant with no memory of past conversations, I don't know what math problem you asked. \n\nPlease ask me your math problem and I'll do my best to solve it! 😊  \n\n"